In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates

R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`


R version 3.6.1 (2019-07-05) -- "Action of the Toes"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.



Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.6.3
│ -- Attaching packages --------------------------------------- tidyverse 1.3.0 --
│ v ggplot2 3.3.0     v purrr   0.3.3
│ v tibble  3.0.0     v stringr 1.4.0
│ v tidyr   1.0.2     v forcats 0.5.0
│ v readr   1.3.1     
│ Warning: package 'ggplot2' was built under R version 3.6.3
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'forc

SimulateDataframe (generic function with 1 method)

In [ ]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
PSE_Difference = 0.1
JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 1
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for n in Range_Participants
        CurrentRunthrough = CurrentRunthrough + 1

        if CurrentRunthrough > 1
        print("This runthrough took ", Dates.now()-TimeStartRunthrough, " s")                
        end
        
        print("Starting runthrough N° ", CurrentRunthrough, " out of ", TotalNumber)
        global TimeStartRunthrough = Dates.now()

        for j in 1:nIterations
            R"""
            ID = paste0("s",1:n)
            
            Psychometric = SimulatePsychometricFunction_Staircase(ID,
                            ConditionOfInterest,
                            StandardValues,
                            1:reps,
                            PSE_Difference,
                            JND_Difference,
                            Multiplicator_PSE_Standard,
                            Multiplicator_SD_Standard,
                            SD_ResponseFunction,
                            Mean_Variability_Between,
                            SD_Variability_Between)

            Psychometric$StandardValues = as.character(Psychometric$StandardValues)

            """
                    
            @rget Psychometric
                    
            TimeStartTrial = Dates.now()
            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            GLMM = fit!(GLMM_Model, fast=true)
            DurationGLMM_NeldMeader_AGP0 = ((Dates.now()) - TimeStartTrial)
            AIC_NeldMeader_AGP0 = aic(GLMM)
            Pvalues_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]   
            
#            TimeStartTrial = Dates.now()
#            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
#            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
#            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
#            GLMM = fit!(GLMM_Model, fast=false)
#            DurationGLMM_NeldMeader_AGP1 = ((Dates.now()) - TimeStartTrial)
#            AIC_NeldMeader_AGP1 = aic(GLMM)
#            Pvalues_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]

            TimeStartTrial = Dates.now()
            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            #default optimizer is bobyqa
            GLMM = fit!(GLMM_Model, fast=true)
            DurationGLMM_bobyqa_AGP0 = ((Dates.now()) - TimeStartTrial)
            AIC_bobyqa_AGP0 = aic(GLMM)
            Pvalues_bobyqa_AGP0 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]
                    
#            TimeStartTrial = Dates.now()
#            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
#            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
#            #default optimizer is bobyqa
#            GLMM = fit!(GLMM_Model, fast=false)
#            DurationGLMM_bobyqa_AGP1 = ((Dates.now()) - TimeStartTrial)
#            AIC_bobyqa_AGP1 = aic(GLMM)
#            Pvalues_bobyqa_AGP1 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]

            if CurrentRunthrough == 1 && j == 1 && reps == Range_reps[1] && n == Range_Participants[1]
                global PowerfulDataframe = DataFrame(
                        nIteration = j,
                        n = n,
                        reps = reps,
                        
                        DurationGLMM_NeldMeader_AGP0=DurationGLMM_NeldMeader_AGP0, 
                        AIC_NeldMeader_AGP0=AIC_NeldMeader_AGP0, 
                        Pvalues_NeldMeader_AGP0_Accuracy=Pvalues_NeldMeader_AGP0[1],
                        Pvalues_NeldMeader_AGP0_Precision=Pvalues_NeldMeader_AGP0[2],
                    
#                        DurationGLMM_NeldMeader_AGP1=DurationGLMM_NeldMeader_AGP1,
#                        AIC_NeldMeader_AGP1=AIC_NeldMeader_AGP1,
#                        Pvalues_NeldMeader_AGP1=Pvalues_NeldMeader_AGP1,
                    
                        DurationGLMM_bobyqa_AGP0=DurationGLMM_bobyqa_AGP0, 
                        AIC_bobyqa_AGP0=AIC_bobyqa_AGP0, 
                        Pvalues_bobyqa_AGP0_Accuracy=Pvalues_bobyqa_AGP0[1],
                        Pvalues_bobyqa_AGP0_Precision=Pvalues_bobyqa_AGP0[2],
                    
#                        DurationGLMM_bobyqa_AGP1=DurationGLMM_bobyqa_AGP1, 
#                        AIC_bobyqa_AGP1=AIC_bobyqa_AGP1, 
#                        Pvalues_bobyqa_AGP1 = Pvalues_bobyqa_AGP1,
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between)

            else
                row = DataFrame(
                        nIteration = j,
                        n = n,
                        reps = reps,
                        
                        DurationGLMM_NeldMeader_AGP0=DurationGLMM_NeldMeader_AGP0, 
                        AIC_NeldMeader_AGP0=AIC_NeldMeader_AGP0,
                        Pvalues_NeldMeader_AGP0_Accuracy=Pvalues_NeldMeader_AGP0[1],
                        Pvalues_NeldMeader_AGP0_Precision=Pvalues_NeldMeader_AGP0[2],
                    
#                        DurationGLMM_NeldMeader_AGP1=DurationGLMM_NeldMeader_AGP1,
#                        AIC_NeldMeader_AGP1=AIC_NeldMeader_AGP1,
#                        Pvalues_NeldMeader_AGP1=Pvalues_NeldMeader_AGP1,
                    
                        DurationGLMM_bobyqa_AGP0=DurationGLMM_bobyqa_AGP0, 
                        AIC_bobyqa_AGP0=AIC_bobyqa_AGP0, 
                        Pvalues_bobyqa_AGP0_Accuracy=Pvalues_bobyqa_AGP0[1],
                        Pvalues_bobyqa_AGP0_Precision=Pvalues_bobyqa_AGP0[2],
                    
#                        DurationGLMM_bobyqa_AGP1=DurationGLMM_bobyqa_AGP1, 
#                        AIC_bobyqa_AGP1=AIC_bobyqa_AGP1, 
#                        Pvalues_bobyqa_AGP1 = Pvalues_bobyqa_AGP1,
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between)
                
                PowerfulDataframe = append!(PowerfulDataframe,row)
            end
        end
    end
end

CSV.write(join(["Durations_Julia.csv"]),PowerfulDataframe)

Starting runthrough N° 1 out of 24Im hereThis runthrough took 36084 milliseconds sStarting runthrough N° 2 out of 24no here1×21 DataFrame
│ Row │ nIteration │ n     │ reps  │ DurationGLMM_NeldMeader_AGP0 │ AIC_NeldMeader_AGP0 │ Pvalues_NeldMeader_AGP0_Accuracy │ Pvalues_NeldMeader_AGP0_Precision │ DurationGLMM_bobyqa_AGP0 │ AIC_bobyqa_AGP0 │ Pvalues_bobyqa_AGP0_Accuracy │ Pvalues_bobyqa_AGP0_Precision │ ConditionsOfInterest │ StandardValue1 │ StandardValue2 │ PSE_Difference │ JND_Difference │ Multiplicator_PSE_Standard │ Multiplicator_SD_Standard │ SD_ResponseFunction │ Mean_Variability_Between │ SD_Variability_Between │
│     │ Int64      │ Int64 │ Int64 │ Millisecond                  │ Float64             │ Float64                          │ Float64                           │ Millisecond              │ Float64         │ Float64                      │ Float64                       │ Int64                │ Int64          │ Int64          │ Float64        │ Float64        │ Int64      

├─────┼────────────┼───────┼───────┼──────────────────────────────┼─────────────────────┼──────────────────────────────────┼───────────────────────────────────┼──────────────────────────┼─────────────────┼──────────────────────────────┼───────────────────────────────┼──────────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────────────────┼───────────────────────────┼─────────────────────┼──────────────────────────┼────────────────────────┤
│ 1   │ 1          │ 18    │ 30    │ 26105 milliseconds           │ 4027.52             │ 0.00397383                       │ 0.000142099                       │ 4364 milliseconds        │ 4024.65         │ 6.50442e-14                  │ 0.000101956                   │ 2                    │ 5              │ 8              │ 0.1            │ 0.2            │ 0                          │ 0.108                     │ 0.1                 │ 0.1                      │ 0.1                    │This runthrough too

│     │ Int64      │ Int64 │ Int64 │ Millisecond                  │ Float64             │ Float64                          │ Float64                           │ Millisecond              │ Float64         │ Float64                      │ Float64                       │ Int64                │ Int64          │ Int64          │ Float64        │ Float64        │ Int64                      │ Float64                   │ Float64             │ Float64                  │ Float64                │
├─────┼────────────┼───────┼───────┼──────────────────────────────┼─────────────────────┼──────────────────────────────────┼───────────────────────────────────┼──────────────────────────┼─────────────────┼──────────────────────────────┼───────────────────────────────┼──────────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────────────────┼───────────────────────────┼─────────────────────┼──────────────────────────┼────────────────────────┤
│ 1   │ 1         

│ 1   │ 1          │ 20    │ 40    │ 39333 milliseconds           │ 4145.43             │ 9.95824e-12                      │ 0.0008292                         │ 4039 milliseconds        │ 4145.43         │ 9.94594e-12                  │ 0.000829116                   │ 2                    │ 5              │ 8              │ 0.1            │ 0.2            │ 0                          │ 0.108                     │ 0.1                 │ 0.1                      │ 0.1                    │This runthrough took 45165 milliseconds sStarting runthrough N° 13 out of 24no here1×21 DataFrame
│ Row │ nIteration │ n     │ reps  │ DurationGLMM_NeldMeader_AGP0 │ AIC_NeldMeader_AGP0 │ Pvalues_NeldMeader_AGP0_Accuracy │ Pvalues_NeldMeader_AGP0_Precision │ DurationGLMM_bobyqa_AGP0 │ AIC_bobyqa_AGP0 │ Pvalues_bobyqa_AGP0_Accuracy │ Pvalues_bobyqa_AGP0_Precision │ ConditionsOfInterest │ StandardValue1 │ StandardValue2 │ PSE_Difference │ JND_Difference │ Multiplicator_PSE_Standard │ Multiplicator_SD_Standa

In [31]:
PowerfulDataframe

,nIteration,n,reps,DurationGLMM_NeldMeader_AGP0,AIC_NeldMeader_AGP0,Pvalues_NeldMeader_AGP0
,Int64,Int64,Int64,Millisec…,Float64,Float64
1,1,10,30,107945 milliseconds,4031.78,1.25237e-16
2,1,10,30,107945 milliseconds,4031.78,2.70689e-6
3,1,12,30,127224 milliseconds,4102.96,4.47504e-12
4,1,12,30,127224 milliseconds,4102.96,0.000571261
5,1,14,30,82952 milliseconds,4172.98,1.18452e-13
6,1,14,30,82952 milliseconds,4172.98,5.80701e-10
7,1,16,30,116023 milliseconds,4052.47,3.51398e-16
8,1,16,30,116023 milliseconds,4052.47,3.38736e-9
9,1,18,30,82655 milliseconds,3975.31,2.48792e-13


123

ArgumentError: ArgumentError: invalid NLopt arguments: zero step size